In [106]:
import pandas as pd
import datetime
from configparser import ConfigParser
from pysimplicate import Simplicate

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [107]:
invoices = sim.to_pandas( sim.invoice_per_year(2021)).fillna({'project_name':'?'})
# Sometimes project is nog filled in but then there's projectS. Take the first one.
invoices['project_name'] = invoices.apply( lambda a: a['projects_0_name'] if a['project_name']=='?' else a['project_name'], axis=1)

# For Momkai and Jungle Minds take the project name as organization name
invoices['organization_name'] = invoices.apply( lambda a: a['project_name'] if a['organization_name'] in ('Momkai','Jungle Minds') else a['organization_name'], axis=1)


In [108]:
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_rows', 500)

invoice_cols = ['invoice_number', 'total_excluding_vat', 'organization_name', 'project_name', 'project_project_number', 'date']
invoice_line_cols = ['revenue_group_label', 'description', 'amount', 'price']
res = []
for index, invoice in invoices.iterrows():
    #if invoice['organization_name'] != 'Capital A':
    #    continue # !!
    rec = {key: invoice[key] for key in invoice_cols}
    for i in range( 130 ):
        colname = f'invoice_lines_{i}_id'
        if colname in invoice and not pd.isnull(invoice[colname]):
            rec = {key: invoice[key] for key in invoice_cols}
            for invoice_line_col in invoice_line_cols:
                colname = f'invoice_lines_{i}_{invoice_line_col}'
                #print( colname, invoice[colname] )
                rec[invoice_line_col] = invoice[colname]
            res += [rec]
dataframe = pd.DataFrame( res ).fillna({'revenue_group_label':'Onbekend'})
#for index, line in dataframe.iterrows():
#    print()
#    print( line )
#    print( float(line['amount']) * float(line['price']) )
dataframe['turnover'] = dataframe.apply(lambda a: float(a['amount']) * float(a['price']), axis=1)
#dataframe.query('revenue_group_label == "Onbekend"')     
dataframe

,invoice_number,total_excluding_vat,organization_name,project_name,project_project_number,date,revenue_group_label,description,amount,price,turnover
0,2021-023,"2,400",Eurowijs - SLA,Eurowijs - SLA,EUR-1,2021-01-01,Omzet hosting,Hosting,6.000000,400.000000,"2,400"
1,2021-022,"4,200",Volksbank - SLA,Volksbank - SLA,VOLK-1,2021-01-01,Omzet hosting,Hosting,6.000000,700.000000,"4,200"
2,2021-024,720,Avero,BCM Tool - SLA,BCM-1,2021-01-01,Omzet hosting,Hosting,6.000000,120.000000,720
3,2021-020,720,Functionals,Functionals - SLA,FUNC-1,2021-01-01,Omzet hosting,Hosting,6.000000,120.000000,720
4,2021-021,"2,880",IDFA,IDFA - SLA,IDFA-1,2021-01-01,Omzet hosting,Hosting,6.000000,480.000000,"2,880"
...,...,...,...,...,...,...,...,...,...,...,...
1994,2021-200,-520,Eurowijs - SLA,Eurowijs - SLA,NaN,2021-02-28,Omzet service,Service (JM) / 24-02-2021 / Fadhlur Zahri / Se...,-1.000000,99.000000,-99
1995,2021-200,-520,Eurowijs - SLA,Eurowijs - SLA,NaN,2021-02-28,Omzet service,Service (JM) / 24-02-2021 / Fadhlur Zahri / Se...,-1.000000,99.000000,-99
1996,2021-200,-520,Eurowijs - SLA,Eurowijs - SLA,NaN,2021-02-28,Omzet service,Service (JM) / 25-02-2021 / Caspar Geerlings /...,-0.500000,99.000000,-50
1997,2021-200,-520,Eurowijs - SLA,Eurowijs - SLA,NaN,2021-02-28,Omzet service,Service (JM) / 25-02-2021 / Fadhlur Zahri / Se...,-0.250000,99.000000,-25


In [109]:
pd.set_option('display.max_rows', 500)
def fltr( a ):
    if a['revenue_group_label'] in ('Omzet development', 'Omzet app development','Omzet concept en design'):
        return 'Omzet projecten'
    if a['revenue_group_label'] in ('Omzet onderhuur'):
        return 'Omzet overig'
    return a['revenue_group_label']


dataframe['revenue_group'] = dataframe.apply( fltr, axis=1 )
dataframe.groupby(['revenue_group','organization_name']).sum(['turnover'])

total_excluding_vat  \
revenue_group     organization_name                                                         
Omzet hosting     Avero                                                               720   
                  BAM - SLA                                                         4,200   
                  BeleefRoutes                                                      1,440   
                  Compare Tax BV.                                                   1,452   
                  De Breij                                                            720   
                  Eurowijs - SLA                                                    2,400   
                  Functionals                                                         720   
                  Havenbedrijf Amsterdam N.V. / Port of Amsterdam                     588   
                  Houthoff                                                            600   
                  IDFA                                                              2,880   
                  IVA driebergen                                                      200   
                  KCP                                                                 720   
                  Kpito                                                               720   
                  Losberger De Boer Holding GmbH                                    2,080   
                  Metroprop                                                           150   
                  NuNotariaat                                                       2,040   
                  Oerol                                                             2,880   
                  Ortec Finance                                                       600   
                  Sprout Money BV                                                   1,022   
                  TMC                                                               3,180   
                  TOPA                                                                720   
                  Thales Nederland B.V.                                             2,700   
                  VVV Ameland                                                       2,880   
                  VVV Schiermonnikoog                                                 600   
                  VVV Terschelling                                                  2,880   
                  VVV Texel                                                         4,320   
                  Van Esch                                                          5,760   
                  Volksbank - SLA                                                   4,200   
                  Worrell & Jetten Accountants en Adviseurs Zoete...                  348   
                  Zoetermeer                                                          105   
Omzet overig      Apple inc.                                                           72   
                  Leaning Technologies B.V.                                         1,175   
                  VVV Terschelling                                                    500   
                  VVV Texel                                                           500   
                  Weiss Legal                                                       2,463   
Omzet projecten   Accell IT                                                    11,589,344   
                  Breens Investments B.V.                                           6,000   
                  Capital A                                                         5,040   
                  Capitola                                                          1,650   
                  Chickslovefood B.V.                                               4,560   
                  EasyBroker                                                        4,590   
                  Havas Lemz                                           

In [110]:
dataframe.query('organization_name=="IDFA"')

,invoice_number,total_excluding_vat,organization_name,project_name,project_project_number,date,revenue_group_label,description,amount,price,turnover,revenue_group
4,2021-021,"2,880",IDFA,IDFA - SLA,IDFA-1,2021-01-01,Omzet hosting,Hosting,6.000000,480.000000,"2,880",Omzet hosting
398,2021-081,360,IDFA,IDFA - SLA,NaN,2021-01-31,Omzet service,Service Plus / 13-01-2021 / Fadhlur Zahri / Se...,1.000000,90.000000,90,Omzet service
399,2021-081,360,IDFA,IDFA - SLA,NaN,2021-01-31,Omzet service,Service Plus / 14-01-2021 / Fadhlur Zahri / Se...,1.000000,90.000000,90,Omzet service
400,2021-081,360,IDFA,IDFA - SLA,NaN,2021-01-31,Omzet service,Service Plus / 14-01-2021 / Fadhlur Zahri / Se...,1.000000,90.000000,90,Omzet service
401,2021-081,360,IDFA,IDFA - SLA,NaN,2021-01-31,Omzet service,Service Plus / 19-01-2021 / Jeroen Soeteman / ...,1.000000,90.000000,90,Omzet service
744,2021-125,"4,945",IDFA,Salesforce afronden,NaN,2021-02-28,Omzet app development,Development Sprint / 26-01-2021 / Miek ten Bru...,1.000000,80.000000,80,Omzet projecten
745,2021-125,"4,945",IDFA,Salesforce afronden,NaN,2021-02-28,Omzet app development,Development Sprint / 26-01-2021 / Jochem Tijhu...,0.500000,95.000000,48,Omzet projecten
746,2021-125,"4,945",IDFA,Salesforce afronden,NaN,2021-02-28,Omzet app development,Development Sprint / 27-01-2021 / Miek ten Bru...,3.000000,80.000000,240,Omzet projecten
747,2021-125,"4,945",IDFA,Salesforce afronden,NaN,2021-02-28,Omzet app development,Development Sprint / 27-01-2021 / Jochem Tijhu...,3.500000,95.000000,332,Omzet projecten
748,2021-125,"4,945",IDFA,Salesforce afronden,NaN,2021-02-28,Omzet app development,Development Sprint / 27-01-2021 / Jeroen Soete...,3.500000,80.000000,280,Omzet projecten


In [115]:
dataframe['turnover'].sum()


792837.855